In [1]:
import pandas as pd
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
import math
import openpyxl
from statistics import * 

In [2]:
#This code will parse through the Peak_ID worksheet and generate another worksheet in the same file called "External_STD"
#with concentration and peak area data for C7, C8, ..., C17 arranged like so:
#
# Chain Length | Conc (mg/L) | Peak Area | Conc (mg/L) | Peak Area | Conc (mg/L) | Peak Area | Conc (mg/L) | Peak Area
# _____________________________________________________________________________________________________________________
#      C7      |    1000     |     #     |     500     |     #     |     100     |     #     |     25      |     #
#      C8      |    1000     |     #     |     500     |     #     |     100     |     #     |     25      |     #
#      ...     |    1000     |     #     |     500     |     #     |     100     |     #     |     25      |     #
#      C17     |    1000     |     #     |     500     |     #     |     100     |     #     |     25      |     #
#
#Peak_ID worksheet lay out...
#


#Add user input for conc

In [5]:
def getExternalStdData(file, sheet_name, area_col, chain_col, alc_acid_ID, saveAs):
    col = [area_col, chain_col]
    wb = openpyxl.load_workbook(file)
    sheet = wb[sheet_name]
    #create dataframe from relevant columns only
    df = pd.read_excel(file, sheet_name=sheet_name, usecols = col, header = None)
    #convert dataframe to array to allow array traversals
    dfList = df.values.tolist()
    #Master_dict will hold area and concentration data 
    Master_dict = {'C3':[],'C4':[],'C5':[],'C6':[],'C7':[], 'C8':[], 'C9':[], 'C10':[], 'C11':[], 'C12':[],
                 'C13':[], 'C14':[], 'C15':[], 'C16':[], 'C17':[]}

    # creates list of external standard title indicies (location of titles)
    alc_acid_indicies = []
    for i in range(len(dfList)):
        name = dfList[i][1]
        if name[0:name.find('-')] == alc_acid_ID:
            alc_acid_indicies.append(i)
    
    conc_array = []
    # start adding to master matrix here using indicies of titles
    for i in alc_acid_indicies:
        #find the position right before the start of a new set of data as indicated by a string
        endindex = i
        for k in range(len(dfList[i:len(dfList)])):
            endindex+=1
            if(isinstance(dfList[endindex][0],str)):
                break
        #parse title to find concentration (gets the number after '-' in the title) and add to conc_array and sort
        conc = dfList[i][1][dfList[i][1].find('-')+1:len(dfList[i][1])]
        conc_array.append(int(conc))
        conc_array.sort()
        #for each chain, add the concentration and area data to the Master dictionary
        for j in range(endindex-i):
            if(isinstance(dfList[i+j][0], int)):
                conc_and_area = [conc, dfList[i+j][0]]
                Master_dict[dfList[i+j][1]].append(conc_and_area)
    
    #create master matrix holding arrays of area values
    num_conc, num_chain = len(conc_array), len(Master_dict);
    Master = [[0 for x in range(num_conc*2+1)] for y in range(num_chain+1)]
    
    #populate column titles
    Master[0][0] = 'Chain Length'
    for i in range(1, num_conc*2+1):
        if i%2==0:
            Master[0][i] = 'Peak Area'
        else:
            Master[0][i] = 'Conc (mg/L)'
    
    #create matrix with desired layout as seen in Cell 2
    chain_iterator = 1
    for chain in Master_dict:
        conc_iterator = 1
        #label first column with chain
        Master[chain_iterator][0] = chain
        for conc in conc_array:
            area = []
            #search for correct area data
            for pair in Master_dict[chain]:
                if int(pair[0]) == conc:
                    area.append(pair[1])
                    break
            #this accounts for double-counting areas (only picks the largest one)
            area.sort(reverse = True)
            #this accounts for no area data
            if len(area) == 0:
                area = ['-']
            #add to the master matrix
            Master[chain_iterator][conc_iterator] = conc
            conc_iterator+=1
            Master[chain_iterator][conc_iterator] = area[0]
            conc_iterator+=1
        chain_iterator+=1
    excelData = pd.DataFrame(data=Master, index=None)
    print(excelData)
    #write to the excel file
    writer = pd.ExcelWriter(file, engine = 'openpyxl')
    writer.book = wb
    excelData.to_excel(writer, sheet_name = "EXT_STD", index = False)
    wb.save(filename = saveAs)
    return excelData

In [7]:
file = 'GCData-JGI_ACRs.xlsx'
sheet_name = 'Peak_ID'
area_col = 3
chain_col = 5
alc_acid_ID = 'FAOH'
saveAs = 'GCData-JGI_ACRs.xlsx'
df2 = getExternalStdData(file, sheet_name, area_col, chain_col, alc_acid_ID, saveAs)

               0            1          2            3          4            5  \
0   Chain Length  Conc (mg/L)  Peak Area  Conc (mg/L)  Peak Area  Conc (mg/L)   
1             C3           10       8532           50      49732          100   
2             C4           10          -           50          -          100   
3             C5           10      10399           50      60722          100   
4             C6           10      11682           50      65225          100   
5             C7           10      12433           50      69285          100   
6             C8           10      12565           50      70574          100   
7             C9           10      14310           50      75654          100   
8            C10           10      13971           50      72999          100   
9            C11           10      15779           50      75524          100   
10           C12           10      14434           50      76281          100   
11           C13           1